In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os # need for file enumeration

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# For reproducibility
np.random.seed(42)

In [2]:
# get our file listings for train and test
train_files = os.listdir('images/train/')
test_files = os.listdir('images/test/')

In [3]:
# convert filenames into dataframes
train_df = pd.DataFrame(train_files, columns = ['filename'])
test_df = pd.DataFrame(test_files, columns = ['filename'])

In [4]:
# label 'em
train_df['hotdog'] = train_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')
test_df['hotdog'] = test_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')

In [5]:
# set up imagedatagenerators
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory = 'images/train',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory = 'images/test',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

Found 3000 validated image filenames belonging to 2 classes.
Found 644 validated image filenames belonging to 2 classes.


In [6]:
# train baseline
train_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [7]:
# test baseline
test_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [8]:
# set up model
cnn_model = Sequential()

cnn_model.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 3) # shape of input (image)
))

cnn_model.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 128, activation = 'relu'))

cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])


In [9]:
# fit model 1 - 5 epochs
cnn_model.fit(
        train_generator,
        epochs = 5,
        validation_data = validation_generator)

Epoch 1/5
94/94 [==============================] - 46s 433ms/step - loss: 0.7233 - accuracy: 0.6330 - val_loss: 0.6001 - val_accuracy: 0.6832
Epoch 2/5
94/94 [==============================] - 40s 423ms/step - loss: 0.5978 - accuracy: 0.6813 - val_loss: 0.5866 - val_accuracy: 0.7050
Epoch 3/5
94/94 [==============================] - 41s 434ms/step - loss: 0.5624 - accuracy: 0.7143 - val_loss: 0.5600 - val_accuracy: 0.7376
Epoch 4/5
94/94 [==============================] - 40s 429ms/step - loss: 0.5509 - accuracy: 0.7150 - val_loss: 0.5537 - val_accuracy: 0.7220
Epoch 5/5
94/94 [==============================] - 41s 441ms/step - loss: 0.5301 - accuracy: 0.7377 - val_loss: 0.5424 - val_accuracy: 0.7314


In [10]:
# fit model 1 - 10 epochs, batch size 128
cnn_model.fit(
        train_generator,
        epochs = 10, batch_size = 128,
        validation_data = validation_generator)

Epoch 1/10
94/94 [==============================] - 41s 433ms/step - loss: 0.5138 - accuracy: 0.7480 - val_loss: 0.5780 - val_accuracy: 0.7252
Epoch 2/10
94/94 [==============================] - 41s 436ms/step - loss: 0.4946 - accuracy: 0.7633 - val_loss: 0.5476 - val_accuracy: 0.7345
Epoch 3/10
94/94 [==============================] - 42s 447ms/step - loss: 0.4706 - accuracy: 0.7757 - val_loss: 0.6530 - val_accuracy: 0.7034
Epoch 4/10
94/94 [==============================] - 43s 459ms/step - loss: 0.4695 - accuracy: 0.7747 - val_loss: 0.6964 - val_accuracy: 0.6817
Epoch 5/10
94/94 [==============================] - 44s 466ms/step - loss: 0.4504 - accuracy: 0.7890 - val_loss: 0.6412 - val_accuracy: 0.7174
Epoch 6/10
94/94 [==============================] - 43s 460ms/step - loss: 0.4336 - accuracy: 0.8000 - val_loss: 0.6428 - val_accuracy: 0.7345
Epoch 7/10
94/94 [==============================] - 41s 436ms/step - loss: 0.4389 - accuracy: 0.7977 - val_loss: 0.7325 - val_accuracy: 0.7065

In [19]:
# set up model
cnn_model_2 = Sequential()

cnn_model_2.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 3) # shape of input (image)
))

cnn_model_2.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model_2.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model_2.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model_2.add(Flatten())

cnn_model_2.add(Dense(units = 128, activation = 'relu'))

cnn_model_2.add(Dense(units = 64, activation = 'relu'))

cnn_model_2.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model_2.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])

In [20]:
# fit model 2 - 5 epochs, batch size 128
cnn_model_2.fit(
        train_generator,
        epochs = 10, batch_size = 128,
        validation_data = validation_generator)

Epoch 1/10
94/94 [==============================] - 62s 655ms/step - loss: 0.7140 - accuracy: 0.5560 - val_loss: 0.6325 - val_accuracy: 0.6320
Epoch 2/10
94/94 [==============================] - 61s 646ms/step - loss: 0.6574 - accuracy: 0.6147 - val_loss: 0.6206 - val_accuracy: 0.6599
Epoch 3/10
94/94 [==============================] - 62s 649ms/step - loss: 0.6313 - accuracy: 0.6397 - val_loss: 0.6604 - val_accuracy: 0.6149
Epoch 4/10
94/94 [==============================] - 63s 665ms/step - loss: 0.6049 - accuracy: 0.6697 - val_loss: 0.6127 - val_accuracy: 0.6755
Epoch 5/10
94/94 [==============================] - 63s 668ms/step - loss: 0.5844 - accuracy: 0.6847 - val_loss: 0.6356 - val_accuracy: 0.6382
Epoch 6/10
94/94 [==============================] - 62s 650ms/step - loss: 0.5737 - accuracy: 0.6943 - val_loss: 0.6580 - val_accuracy: 0.6739
Epoch 7/10
94/94 [==============================] - 61s 641ms/step - loss: 0.5616 - accuracy: 0.7117 - val_loss: 0.6190 - val_accuracy: 0.6724

In [18]:
# same as first one except early stopping

es = EarlyStopping(patience = 5)

# set up model
cnn_model_3 = Sequential()

cnn_model_3.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 3) # shape of input (image)
))

cnn_model_3.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model_3.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model_3.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model_3.add(Flatten())

cnn_model_3.add(Dense(units = 128, activation = 'relu', kernel_regularizer = l2(0.01)))

cnn_model_3.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model_3.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])


In [19]:
# fit model 1 w/es - up to  50 epochs, batch size 64
cnn_model_3.fit(
        train_generator,
        epochs = 50, batch_size = 128,
        validation_data = validation_generator, callbacks = [es])

Epoch 1/50
94/94 [==============================] - 44s 454ms/step - loss: 1.1490 - accuracy: 0.5930 - val_loss: 0.7824 - val_accuracy: 0.6009
Epoch 2/50
94/94 [==============================] - 43s 459ms/step - loss: 0.7186 - accuracy: 0.6473 - val_loss: 0.7237 - val_accuracy: 0.6584
Epoch 3/50
94/94 [==============================] - 41s 431ms/step - loss: 0.6986 - accuracy: 0.6650 - val_loss: 0.7178 - val_accuracy: 0.6382
Epoch 4/50
94/94 [==============================] - 41s 434ms/step - loss: 0.6847 - accuracy: 0.6627 - val_loss: 0.6773 - val_accuracy: 0.6832
Epoch 5/50
94/94 [==============================] - 40s 428ms/step - loss: 0.6682 - accuracy: 0.6797 - val_loss: 0.6462 - val_accuracy: 0.7019
Epoch 6/50
94/94 [==============================] - 42s 443ms/step - loss: 0.6627 - accuracy: 0.6780 - val_loss: 0.6588 - val_accuracy: 0.7081
Epoch 7/50
94/94 [==============================] - 42s 442ms/step - loss: 0.6590 - accuracy: 0.6877 - val_loss: 0.6527 - val_accuracy: 0.6832

In [29]:
cnn_model_3.save('./models/cnn_3.ksm')

INFO:tensorflow:Assets written to: ./models/cnn_3.ksm\assets


INFO:tensorflow:Assets written to: ./models/cnn_3.ksm\assets
